In [1]:
import pandas as pd
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
from collections import defaultdict
from multiprocessing import  Pool
import numpy as np
import swifter
import pandas
from pandarallel import pandarallel

In [2]:
FILE_ORIGIN = '../dblpv13.json'
FILE_PREPROCESSED = '../data.json'

In [3]:
CHUNKSIZE = 100000
MAX_OBSERVATIONS = 1000000 #Set to 55000000 if want all rows
df = pd.DataFrame()
with pd.read_json(FILE_PREPROCESSED, orient='records', lines=True, chunksize=CHUNKSIZE, nrows=MAX_OBSERVATIONS) as reader:
    for chunk_i, chunk in tqdm_notebook(enumerate(reader), total=MAX_OBSERVATIONS // CHUNKSIZE):
        df_new = pd.DataFrame(chunk)
        df_new['chunk_idx'] = chunk_i
        df = pd.concat([df, df_new], ignore_index=True)

  0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
df.head()

,_id,title,venue,year,keywords,n_citation,lang,authors,fos,page_start,...,volume,issue,issn,isbn,doi,pdf,url,abstract,references,chunk_idx
0,53e99784b7602d9701f3e3f5,3GIO.,{'type': 0},2011.0,[],0.0,en,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,53e99784b7602d9701f3e133,The relationship between canopy parameters and...,"{'_id': '53a7297d20f7420be8bd4ae7', 'name_d': ...",2011.0,"[canopy parameters, canopy spectrum, different...",0.0,en,"[{'_id': '53f45728dabfaec09f209538', 'name': '...","[Agronomy, Moisture, Hydrology, Environmental ...",1930,...,null,null,,,10.1109/IGARSS.2011.6049503,None,[http://dx.doi.org/10.1109/IGARSS.2011.6049503],Drought is the first place in all the natural ...,NaN,0
2,53e99784b7602d9701f3e151,A solution to the problem of touching and brok...,"{'_id': '53a72a4920f7420be8bfa51b', 'name_d': ...",1993.0,"[handwriting recognition, prototypes, image se...",17.0,en,"[{'_id': '53f46797dabfaeb22f542630', 'name': '...","[Intelligent character recognition, Pattern re...",602,...,,,,,10.1109/ICDAR.1993.395663,None,[http://dx.doi.org/10.1109/ICDAR.1993.395663],,"[53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990c...",0
3,53e99784b7602d9701f3e15d,Timing yield estimation using statistical stat...,"{'_id': '53a72e2020f7420be8c80142', 'name_d': ...",2005.0,"[sequential circuits, statistical distribution...",28.0,en,"[{'_id': '53f43b03dabfaedce555bf2a', 'name': '...","[Delay calculation, Timing failure, Monte Carl...",2461,...,,,,0-7803-8834-8,10.1109/ISCAS.2005.1465124,//static.aminer.org/pdf/PDF/000/423/329/timing...,"[http://dx.doi.org/10.1109/ISCAS.2005.1465124,...",As process variations become a significant pro...,"[53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27...",0
4,53e99784b7602d9701f3e161,360°,"{'_id': '5390a74a20f70186a0e8b40b', 'raw': 'AC...",2009.0,"[global high technology, daily short-distance ...",NaN,en,"[{'_id': '53f46946dabfaec09f24b4ed', 'name': '...",NaN,39,...,,,,,10.1145/1665137.1665166,NaN,NaN,360° represents the concerns that are addresse...,NaN,0


In [5]:
articles_cited = defaultdict(int)
for val in df.references:
    if isinstance(val, list):
        for art in val:
            articles_cited[art] += 1

In [6]:
df_cited = pd.DataFrame([item for item in articles_cited.items()], columns=['_id', 'n_cited'])
df_cited.head()

,_id,n_cited
0,53e99cf5b7602d97025ace63,10
1,557e8a7a6fee0fe990caa63d,35
2,53e9a96cb7602d97032c459a,156
3,53e9b929b7602d9704515791,24
4,557e59ebf6678c77ea222447,30


In [7]:
df = pd.merge(df, df_cited, on='_id', how='left')

In [8]:
df.head()

,_id,title,venue,year,keywords,n_citation,lang,authors,fos,page_start,...,issue,issn,isbn,doi,pdf,url,abstract,references,chunk_idx,n_cited
0,53e99784b7602d9701f3e3f5,3GIO.,{'type': 0},2011.0,[],0.0,en,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,53e99784b7602d9701f3e133,The relationship between canopy parameters and...,"{'_id': '53a7297d20f7420be8bd4ae7', 'name_d': ...",2011.0,"[canopy parameters, canopy spectrum, different...",0.0,en,"[{'_id': '53f45728dabfaec09f209538', 'name': '...","[Agronomy, Moisture, Hydrology, Environmental ...",1930,...,null,,,10.1109/IGARSS.2011.6049503,None,[http://dx.doi.org/10.1109/IGARSS.2011.6049503],Drought is the first place in all the natural ...,NaN,0,NaN
2,53e99784b7602d9701f3e151,A solution to the problem of touching and brok...,"{'_id': '53a72a4920f7420be8bfa51b', 'name_d': ...",1993.0,"[handwriting recognition, prototypes, image se...",17.0,en,"[{'_id': '53f46797dabfaeb22f542630', 'name': '...","[Intelligent character recognition, Pattern re...",602,...,,,,10.1109/ICDAR.1993.395663,None,[http://dx.doi.org/10.1109/ICDAR.1993.395663],,"[53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990c...",0,1.0
3,53e99784b7602d9701f3e15d,Timing yield estimation using statistical stat...,"{'_id': '53a72e2020f7420be8c80142', 'name_d': ...",2005.0,"[sequential circuits, statistical distribution...",28.0,en,"[{'_id': '53f43b03dabfaedce555bf2a', 'name': '...","[Delay calculation, Timing failure, Monte Carl...",2461,...,,,0-7803-8834-8,10.1109/ISCAS.2005.1465124,//static.aminer.org/pdf/PDF/000/423/329/timing...,"[http://dx.doi.org/10.1109/ISCAS.2005.1465124,...",As process variations become a significant pro...,"[53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27...",0,2.0
4,53e99784b7602d9701f3e161,360°,"{'_id': '5390a74a20f70186a0e8b40b', 'raw': 'AC...",2009.0,"[global high technology, daily short-distance ...",NaN,en,"[{'_id': '53f46946dabfaec09f24b4ed', 'name': '...",NaN,39,...,,,,10.1145/1665137.1665166,NaN,NaN,360° represents the concerns that are addresse...,NaN,0,NaN


In [9]:
import pickle
from model import Model
from coauthors_model import CoAuthors

with open('model', 'rb') as picklefile:
    clustering = pickle.load(picklefile)

with open('coauthors_model', 'rb') as picklefile:
    co_authors = pickle.load(picklefile)

In [10]:
df['title_abst'] = df['title'] + ' ' + df['abstract']

In [11]:
def get_topic(x):
    if x is None or not isinstance(x, str) or len(x) < 2:
        return -1
    else:
        res = clustering.predict([x])
        return res[0][0]

In [12]:
pandarallel.initialize(progress_bar=True)
df['topic'] = df.title.parallel_apply(get_topic)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [13]:
df.head()

,_id,title,venue,year,keywords,n_citation,lang,authors,fos,page_start,...,isbn,doi,pdf,url,abstract,references,chunk_idx,n_cited,title_abst,topic
0,53e99784b7602d9701f3e3f5,3GIO.,{'type': 0},2011.0,[],0.0,en,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0
1,53e99784b7602d9701f3e133,The relationship between canopy parameters and...,"{'_id': '53a7297d20f7420be8bd4ae7', 'name_d': ...",2011.0,"[canopy parameters, canopy spectrum, different...",0.0,en,"[{'_id': '53f45728dabfaec09f209538', 'name': '...","[Agronomy, Moisture, Hydrology, Environmental ...",1930,...,,10.1109/IGARSS.2011.6049503,None,[http://dx.doi.org/10.1109/IGARSS.2011.6049503],Drought is the first place in all the natural ...,NaN,0,NaN,The relationship between canopy parameters and...,6
2,53e99784b7602d9701f3e151,A solution to the problem of touching and brok...,"{'_id': '53a72a4920f7420be8bfa51b', 'name_d': ...",1993.0,"[handwriting recognition, prototypes, image se...",17.0,en,"[{'_id': '53f46797dabfaeb22f542630', 'name': '...","[Intelligent character recognition, Pattern re...",602,...,,10.1109/ICDAR.1993.395663,None,[http://dx.doi.org/10.1109/ICDAR.1993.395663],,"[53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990c...",0,1.0,A solution to the problem of touching and brok...,8
3,53e99784b7602d9701f3e15d,Timing yield estimation using statistical stat...,"{'_id': '53a72e2020f7420be8c80142', 'name_d': ...",2005.0,"[sequential circuits, statistical distribution...",28.0,en,"[{'_id': '53f43b03dabfaedce555bf2a', 'name': '...","[Delay calculation, Timing failure, Monte Carl...",2461,...,0-7803-8834-8,10.1109/ISCAS.2005.1465124,//static.aminer.org/pdf/PDF/000/423/329/timing...,"[http://dx.doi.org/10.1109/ISCAS.2005.1465124,...",As process variations become a significant pro...,"[53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27...",0,2.0,Timing yield estimation using statistical stat...,22
4,53e99784b7602d9701f3e161,360°,"{'_id': '5390a74a20f70186a0e8b40b', 'raw': 'AC...",2009.0,"[global high technology, daily short-distance ...",NaN,en,"[{'_id': '53f46946dabfaec09f24b4ed', 'name': '...",NaN,39,...,,10.1145/1665137.1665166,NaN,NaN,360° represents the concerns that are addresse...,NaN,0,NaN,360° 360° represents the concerns that are add...,0


In [14]:
df[['_id', 'topic']].to_csv('topics.csv', index=False)

In [15]:
df.topic.value_counts()

 19    48954
 37    44807
 1     39062
 34    38602
 32    38118
 39    36548
 41    34602
 33    32956
 16    32771
 6     32391
 26    31419
 2     30439
 17    30081
 21    30070
 4     29612
 5     29502
 3     29238
 30    26568
 25    26353
 9     26188
 20    25069
 27    24920
 35    22359
 12    22252
 8     19618
 7     19192
 11    17596
 10    17276
 14    16677
 36    16485
 18    16240
 22    15216
 15    14464
 23    11765
 31    11604
 13    11281
 40    10935
 0      9981
 29     9247
 28     8982
 38     7495
 24     2961
-1       104
Name: topic, dtype: int64

In [ ]:
authors = set(co_authors.association_rules.antecedents.apply(lambda x: list(x)[0]))

In [16]:
authors_articles = defaultdict(set)
for row in df[['_id', 'authors']].itertuples():
    if isinstance(row.authors, list):
        for author in row.authors:
            if 'name' in author and author['name'] in authors:
                authors_articles[author['name']].add(row._1)

NameError: name 'authors' is not defined

In [ ]:
class RecommendationArticles:
    def __init__(self, clustering, co_authors, db, authors_articles, top_n):
        self.clustering = clustering
        self.co_authors = co_authors
        self.db = db
        self.authors_articles = authors_articles
        self.top_n = top_n
        self.columns = ['title', 'year', 'doi']

    def predict(self, text=None, author=None):
        if text and author:
            result = self._get_prediction_text_author(text, author, self.top_n, self.db)
            if len(result) < self.top_n:
                if len(result) == 0:
                    new_db = self.db
                else:
                    new_db = self.db[~self.db._id.isin(result._id)]
                result_text = self._get_prediction_text(text, self.top_n - len(result), new_db)
                result = pd.concat([result, result_text], ignore_index=True)
        elif text:
            result = self._get_prediction_text(text, self.top_n, self.db)
        else:
            result = self._get_prediction_author(text, self.top_n, self.db)
        if result.empty:
            return result.to_json(orient='records')
        return result[self.columns].to_json(orient='records')
                
    def _get_prediction_text_author(self, text, author, top_n, db):
        topic = self.clustering.predict([text])[0][0]
        co_authors = self.co_authors.predict(author)
        result = pd.DataFrame()
        if co_authors and topic:
            for co_author in co_authors:
                if co_author in self.authors_articles:
                    temp_df = db.loc[db._id.isin(self.authors_articles[co_author]) & db.topic.eq(topic)]
                    result = pd.concat([result, temp_df], ignore_index=True)
        if result.empty:
            return result
        return result.nlargest(top_n, 'n_cited')

    def _get_prediction_text(self, text, top_n, db):
        topic = self.clustering.predict([text])[0][0]
        result = db.loc[db.topic.eq(topic)]
        if result.empty:
            return result
        return result.nlargest(top_n, 'n_cited')

    def _get_prediction_author(self, author, top_n, db):
        co_authors = self.co_authors.predict(author)
        result = pd.DataFrame()
        for co_author in co_authors:
            if co_author in self.authors_articles:
                temp_df = db.loc[db._id.isin(self.authors_articles[co_author])]
                result = pd.concat([result, temp_df], ignore_index=True)
        if result.empty:
            return result
        return result.nlargest(top_n, 'n_cited')

In [ ]:
recommendation_articles = RecommendationArticles(clustering, co_authors, df[['_id', 'title', 'year', 'doi', 'n_cited', 'topic']], authors_articles, 10)

In [ ]:
recommendation_articles.predict('Machine Learning, Deep LEarning', 'Yunquan Zhang')

In [ ]:
import pickle
with open('articles_model', 'wb') as picklefile:
    pickle.dump(recommendation_articles, picklefile)